# 전이학습

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import Input, models, layers, optimizers, metrics
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.applications import VGG16
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 학습셋의 변형을 설정하는 부분입니다.- 이미지 생성 옵션
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                  #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                  #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                  #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                  #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                  )

# 실제 데이터가 있는 곳을 알려주고 이미지를 불러옴
train_generator = train_datagen.flow_from_directory(
       './image_data/train',   # 학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

Found 160 images belonging to 2 classes.


In [4]:
# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255) # 정규화만 수행

test_generator = test_datagen.flow_from_directory(
       './image_data/test',      # 테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),  # 이미지 크기
       batch_size=5,
       class_mode='binary')     # 치매 / 정상 이진 분류 이기 때문에 바이너리 모드로 실행

Found 120 images belonging to 2 classes.


**VGG16**
- 이미지 분류 문제를 위한 딥러닝 모델 중 하나로, 옥스퍼드 대학교 연구팀에서 개발한 모델입니다. VGG16은 대규모 이미지 데이터셋인 ImageNet에서 사전 학습된 가중치를 활용하여 다양한 이미지 분류 작업에 유용하게 사용할 수 있습니다.
  - weights='imagenet': 
    - 이 매개변수는 VGG16 모델의 가중치를 어떤 값으로 초기화할지를 결정합니다. 'imagenet'으로 설정하면 ImageNet 데이터셋에서 미리 학습한 가중치를 사용합니다. 이를 통해 모델이 이미지 분류 문제를 더 쉽게 학습할 수 있습니다.

  - include_top=False: 
    - 이 매개변수는 VGG16의 최상위(fully connected) 계층(top)을 포함할지 여부를 결정합니다. True로 설정하면 VGG16 모델의 마지막 계층까지 포함하여 이미지 분류를 수행하는 모델이 생성되지만, False로 설정하면 VGG16의 최상위 계층을 제외한 부분만 생성됩니다. 주로 전이 학습(transfer learning)을 위해 사용되며, 사용자가 자신만의 분류기(classifier)를 추가하거나 다른 작업에 활용할 수 있도록 합니다.

  - input_shape=(150, 150, 3): 
    - 입력 이미지의 크기를 정의하는 매개변수입니다. VGG16 모델은 입력으로 150x150 크기의 컬러(RGB) 이미지를 기대합니다. 따라서 이를 입력으로 사용하여 모델을 생성하게 됩니다.

In [5]:
# VGG16 모델을 불러옵니다.
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False
transfer_model.summary()

58889256/58889256 [==============================] - 9s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [6]:
# 모델 설정
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64))
finetune_model.add(Activation('relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(1))
finetune_model.add(Activation('sigmoid'))
finetune_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
 activation_1 (Activation)   (None, 1)                 0

In [7]:
# 모델의 실행 옵션을 설정합니다.
finetune_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# 모델을 실행합니다.
history = finetune_model.fit(
       train_generator,
       epochs=20,
       validation_data=test_generator,
       validation_steps=10,
       callbacks=[early_stopping_callback])

Epoch 1/20
32/32 [==============================] - 6s 154ms/step - loss: 0.7112 - accuracy: 0.5562 - val_loss: 0.5465 - val_accuracy: 0.8000
Epoch 2/20
32/32 [==============================] - 5s 146ms/step - loss: 0.4903 - accuracy: 0.7500 - val_loss: 0.4488 - val_accuracy: 0.8400
Epoch 3/20
32/32 [==============================] - 5s 148ms/step - loss: 0.4428 - accuracy: 0.8125 - val_loss: 0.4028 - val_accuracy: 0.8200
Epoch 4/20
32/32 [==============================] - 5s 148ms/step - loss: 0.3890 - accuracy: 0.8375 - val_loss: 0.3640 - val_accuracy: 0.8400
Epoch 5/20
32/32 [==============================] - 5s 154ms/step - loss: 0.3682 - accuracy: 0.8562 - val_loss: 0.3427 - val_accuracy: 0.8800
Epoch 6/20
32/32 [==============================] - 5s 152ms/step - loss: 0.2853 - accuracy: 0.9250 - val_loss: 0.2319 - val_accuracy: 0.9000
Epoch 7/20
32/32 [==============================] - 5s 157ms/step - loss: 0.2881 - accuracy: 0.9062 - val_loss: 0.2258 - val_accuracy: 0.9600
Epoch 

In [ ]:
# 검증셋과 학습셋의 오차를 저장합니다.
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 그래프로 표현해 봅니다.
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블을 표시하겠습니다.
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()